In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes(balanced=True)
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.15)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 4.90 sec - Loss 0.060898 - ACC 60.75% - ACC Mean 60.75% - AUC 65.72% - AUC Mean 65.72% - Deter 000
Ite 00050 - 0.95 sec - Loss 0.012384 - ACC 62.55% - ACC Mean 60.69% - AUC 65.21% - AUC Mean 64.84% - Deter 015
Ite 00085 - 0.98 sec - Loss 0.007724 - ACC 58.95% - ACC Mean 60.37% - AUC 63.48% - AUC Mean 64.34% - Deter 050
Early stop ite 85, rollback to correction of ite 35, whith acc of 62.65% and auc of 66.95%
OUR METHOD RUN 1/10 - acc: 62.65% +- 0.0% - auc: 66.95% +- 0.0%
Ite 00000 - 1.02 sec - Loss 0.138130 - ACC 59.75% - ACC Mean 59.75% - AUC 65.04% - AUC Mean 65.04% - Deter 000
Ite 00050 - 1.24 sec - Loss 0.052408 - ACC 60.65% - ACC Mean 61.02% - AUC 64.90% - AUC Mean 65.16% - Deter 031
Ite 00069 - 1.07 sec - Loss 0.042984 - ACC 60.20% - ACC Mean 60.73% - AUC 64.65% - AUC Mean 64.95% - Deter 050
Early stop ite 69, rollback to correction of ite 19, whith acc of 61.95% and auc of 66.91%
OUR METHOD RUN 2/10 - acc: 61.95% +- 0.0% - auc: 66.91% +- 0.0%
Ite 00000 - 0.98 sec - L

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 60.65% +- 0.0% - auc: 64.04% +- 0.0%
MEAN - acc: 58.75% +- 0.0% - auc: 63.8% +- 0.0%
KNN - acc: 60.55% +- 0.0% - auc: 64.29% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:43<00:00, 114.80it/s]



GAIN RUN 1/10 - acc: 59.05% +- 1.63% - auc: 61.93% +- 2.39%



100%|██████████| 5000/5000 [00:44<00:00, 113.10it/s]



GAIN RUN 2/10 - acc: 60.05% +- 2.36% - auc: 64.14% +- 2.79%



100%|██████████| 5000/5000 [00:42<00:00, 117.53it/s]


...


GAIN RUN 8/10 - acc: 59.65% +- 2.31% - auc: 63.96% +- 2.65%



100%|██████████| 5000/5000 [00:48<00:00, 103.52it/s]



GAIN RUN 9/10 - acc: 60.0% +- 1.65% - auc: 63.68% +- 2.48%



100%|██████████| 5000/5000 [00:49<00:00, 101.96it/s]



GAIN RUN 10/10 - acc: 61.4% +- 2.24% - auc: 65.48% +- 3.48%

GAIN GLOBAL - acc: 60.2% +- 0.73% - auc: 64.4% +- 1.11%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 61.2% +- 0.0% - auc: 64.88% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1

MISSFOREST RUN 1/10 - acc: 59.65% +- 2.12% - auc: 64.29% +- 2.59%

Iteration: 0
Iteration: 1

MISSFOREST RUN 2/10 - acc: 60.4% +- 0.94% - auc: 64.32% +- 1.92%

Iteration: 0
Iteration: 1

MISSFOREST RUN 3/10 - acc: 62.15% +- 1.09% - auc: 65.36% +- 2.72%

Iteration: 0
Iteration: 1

MISSFOREST RUN 4/10 - acc: 61.9% +- 1.56% - auc: 65.79% +- 2.22%

Iteration: 0
Iteration: 1

MISSFOREST RUN 5/10 - acc: 60.65% +- 1.29% - auc: 64.71% +- 2.2%

Iteration: 0
Iteration: 1

MISSFOREST RUN 6/10 - acc: 59.8% +- 1.45% - auc: 64.5% +- 2.07%

Iteration: 0
Iteration: 1

MISSFOREST RUN 7/10 - acc: 60.15% +- 3.23% - auc: 63.84% +- 3.46%

Iteration: 0
Iteration: 1

MISSFOREST RUN 8/10 - acc: 61.45% +- 1.94% - auc: 66.07% +- 2.3%

Iteration: 0
Iteration: 1

MISSFOREST RUN 9/10 - acc: 62.35% +- 1.35% - auc: 66.07% +- 3.13%

Iteration: 0
Iteration: 1

MISSFOREST RUN 10/10 - acc: 60.9% +- 2.77% - auc: 65.45% +- 2.68%


MISSFOREST GLOBAL - acc: 60.94% +- 0.93% - auc: 65.04% +- 0.77%


## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 61.75% +- 2.16% - auc: 65.12% +- 3.32%
SOFTIMPUTE RUN 2/10 - acc: 58.8% +- 0.4% - auc: 63.73% +- 0.69%
SOFTIMPUTE RUN 3/10 - acc: 61.05% +- 2.61% - auc: 64.56% +- 2.98%
SOFTIMPUTE RUN 4/10 - acc: 58.95% +- 1.65% - auc: 64.12% +- 2.07%
SOFTIMPUTE RUN 5/10 - acc: 61.3% +- 1.67% - auc: 65.29% +- 1.91%
SOFTIMPUTE RUN 6/10 - acc: 60.8% +- 2.38% - auc: 64.6% +- 2.79%
SOFTIMPUTE RUN 7/10 - acc: 59.85% +- 2.41% - auc: 63.56% +- 3.15%
SOFTIMPUTE RUN 8/10 - acc: 60.5% +- 1.64% - auc: 64.54% +- 1.58%
SOFTIMPUTE RUN 9/10 - acc: 60.35% +- 3.12% - auc: 65.1% +- 4.24%
SOFTIMPUTE RUN 10/10 - acc: 60.05% +- 2.43% - auc: 64.18% +- 2.6%

SOFTIMPUTE GLOBAL - acc: 60.34% +- 0.91% - auc: 64.48% +- 0.56%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4094

SINKHORN RUN 1/10 - acc: 59.95% +- 2.81% - auc: 64.1% +- 2.72%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4365

SINKHORN RUN 2/10 - acc: 58.8% +- 2.34% - auc: 63.4% +- 2.0%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4261

SINKHORN RUN 3/10 - acc: 57.8% +- 2.22% - auc: 62.75% +- 2.36%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.5577

SINKHORN RUN 4/10 - acc: 58.55% +- 2.13% - auc: 62.32% +- 1.94%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4567

SINKHORN RUN 5/10 - acc: 60.8% +- 1.59% - auc: 64.75% +- 1.66%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4533

SINKHORN RUN 6/10 - acc: 60.05% +- 3.44% - auc: 63.45% +- 3.48%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4107

SINKHORN RUN 7/10 - acc: 60.55% +- 1.96% - auc: 63.07% +- 2.73%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.5078

SINKHORN RUN 8/10 - acc: 58.9% +- 2.93% - auc: 62.35% +

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 61.45% +- 2.2% - auc: 65.15% +- 1.3%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 60.85% +- 2.74% - auc: 64.99% +- 3.76%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 60.3% +- 1.49% - auc: 64.23% +- 2.35%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 59.3% +- 1.76% - auc: 63.96% +- 1.91%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 60.5% +- 2.07% - auc: 64.74% +- 1.9%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 59.7% +- 2.81% - auc: 65.49% +- 3.85%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 59.65% +- 1.87% - auc: 63.71% +- 1.87%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 58.25% +- 1.89% - auc: 61.86% +- 1.33%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 59.

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC -